# Práctica 1 - Regresión

Vamos a trabajar con un conjunto de datos que representa una serie de billetes de renfe. Entre sus parámetros se encuentran:




*   **id**: identificador del billete
*   **company**: empresa
*   **origin**: lugar de salida del tren
*   **destination**: lugar de llegada del tren
*   **departure**: hora de salida del tren
*   **arrival**: hora de llegada del tren
*   **duration**: duración del trayecto
*   **vehicle_type**: tipo de tren
*   **vehicle_class**: clase del pasajero
*   **price**: precio (variable objetivo, a predecir)





In [47]:
test_url = "https://gist.githubusercontent.com/w-dan/a3c63b2aed66a5edd1288c1d3006fc79/raw/9dcc831af941428edad3ef285729e4ef5a63a9fb/renfe-test.csv"
train_url = "https://gist.githubusercontent.com/w-dan/a3c63b2aed66a5edd1288c1d3006fc79/raw/9dcc831af941428edad3ef285729e4ef5a63a9fb/renfe-train.csv"

In [48]:
import pandas as pd
train_df = pd.read_csv(train_url)
test_df = pd.read_csv(test_url)

In [49]:
test_df

,id,company,origin,destination,departure,arrival,duration,vehicle_type,vehicle_class,price,fare,seats,meta,insert_date
0,3002,renfe,MADRID,BARCELONA,2019-04-28 08:30:00,2019-04-28 11:15:00,2.75,AVE,Turista,75.40,Promo,NaN,{},2019-04-11 22:00:09
1,3003,renfe,MADRID,BARCELONA,2019-04-28 20:00:00,2019-04-28 22:30:00,2.50,AVE,Preferente,115.65,Promo,NaN,{},2019-04-11 22:00:09
2,3004,renfe,MADRID,BARCELONA,2019-04-28 09:30:00,2019-04-28 12:34:00,3.07,AVE,Turista Plus,90.50,Promo,NaN,{},2019-04-11 22:00:09
3,3005,renfe,MADRID,VALENCIA,2019-06-03 12:21:00,2019-06-03 19:04:00,6.72,REGIONAL,Turista,28.35,Adulto ida,NaN,{},2019-04-11 22:00:09
4,3006,renfe,MADRID,BARCELONA,2019-04-28 10:30:00,2019-04-28 13:15:00,2.75,AVE,Turista,85.10,Promo,NaN,{},2019-04-11 22:00:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,4996,renfe,MADRID,BARCELONA,2019-04-12 10:30:00,2019-04-12 13:15:00,2.75,AVE,Turista,107.70,Flexible,NaN,{},2019-04-11 22:07:27
1995,4997,renfe,MADRID,BARCELONA,2019-04-12 16:30:00,2019-04-12 19:15:00,2.75,AVE,Turista,107.70,Flexible,NaN,{},2019-04-11 22:07:27
1996,4998,renfe,MADRID,BARCELONA,2019-04-12 11:30:00,2019-04-12 14:40:00,3.17,AVE,Turista,107.70,Flexible,NaN,{},2019-04-11 22:07:27
1997,4999,renfe,MADRID,BARCELONA,2019-04-12 17:30:00,2019-04-12 20:40:00,3.17,AVE,Turista,NaN,Flexible,NaN,{},2019-04-11 22:07:27


In [50]:
submission = test_df[['id']].copy()

train_df = train_df.drop(['id', 'company', 'seats', 'meta', 'origin', 'insert_date', 'departure', 'arrival', 'vehicle_class', 'fare'], axis=1)
test_df = test_df.drop(['id', 'company', 'seats', 'meta', 'origin', 'insert_date', 'departure', 'arrival', 'vehicle_class', 'fare'], axis=1)

train_df

,destination,duration,vehicle_type,price
0,BARCELONA,3.08,AVE,68.95
1,BARCELONA,2.98,AVE-TGV,107.70
2,BARCELONA,2.83,AVE,75.40
3,BARCELONA,3.17,AVE,NaN
4,BARCELONA,2.50,AVE,106.75
...,...,...,...,...
2995,VALENCIA,6.70,REGIONAL,28.35
2996,BARCELONA,2.85,AVE,107.70
2997,BARCELONA,3.15,AVE,49.55
2998,BARCELONA,3.17,AVE,85.10


In [51]:
from sklearn.preprocessing import OrdinalEncoder
train_df['vehicle_type'] = OrdinalEncoder().fit_transform(train_df[['vehicle_type']])
train_df['destination'] = OrdinalEncoder().fit_transform(train_df[['destination']])

test_df['vehicle_type'] = OrdinalEncoder().fit_transform(test_df[['vehicle_type']])
test_df['destination'] = OrdinalEncoder().fit_transform(test_df[['destination']])

test_df

,destination,duration,vehicle_type,price
0,0.0,2.75,2.0,75.40
1,0.0,2.50,2.0,115.65
2,0.0,3.07,2.0,90.50
3,2.0,6.72,9.0,28.35
4,0.0,2.75,2.0,85.10
...,...,...,...,...
1994,0.0,2.75,2.0,107.70
1995,0.0,2.75,2.0,107.70
1996,0.0,3.17,2.0,107.70
1997,0.0,3.17,2.0,NaN


In [52]:
from sklearn.linear_model import LinearRegression

y = train_df['price']
y_test = test_df['price']

y = y.fillna(y.mean())
y_test = y_test.fillna(y_test.mean())

X_test = test_df.drop(['price'], axis=1)
X = train_df.drop(['price'], axis=1)

reg = LinearRegression().fit(X, y)

In [57]:
pred = reg.predict(X_test)
pred_train = reg.predict(X)

from sklearn.metrics import mean_absolute_error


print("MAE (train): ", mean_absolute_error(y_test, pred))
print("MAE (train): ", mean_absolute_error(y, pred_train))

MAE (train):  12.736846154777966
MAE (train):  12.55922752691473


In [58]:
submission = submission.reindex(columns = submission.columns.tolist() + ['Price'])
submission['Price'] = pred                                                                 

submission

,id,Price
0,3002,83.554961
1,3003,83.948783
2,3004,83.050868
3,3005,27.963430
4,3006,83.554961
...,...,...
1994,4996,83.554961
1995,4997,83.554961
1996,4998,82.893340
1997,4999,82.893340


In [60]:
from google.colab import files

# Exportamos el CSV borrando el índice
submission.to_csv('submission_kaggle.csv', index=False) 
files.download('submission_kaggle.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>